# 13장. 텍스트마이닝_감성분석과 토픽분석

### - pandas 버전이 1.1.4 이상이면서 2.0.0보다 작은지 확인하고, 아니라면 pandas 재설치 하기. (pyLDA 에러 때문)

In [ ]:
import pandas as pd
pd.show_versions()

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.10.12.final.0
python-bits      : 64
OS               : Linux
OS-release       : 5.15.107+
Version          : #1 SMP Sat Apr 29 09:15:28 UTC 2023
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.5.3
numpy            : 1.22.4
pytz             : 2022.7.1
dateutil         : 2.8.2
setuptools       : 67.7.2
pip              : 23.1.2
Cython           : 0.29.34
pytest           : 7.2.2
hypothesis       : None
sphinx           : 3.5.4
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.9.2
html5lib         : 1.1
pymysql          : None
psycopg2         : 2.9.6
jinja2           : 3.1.2
IPython          : 7.34.0
pandas_datareader: 0.10.0
bs4              : 4.11.2
bottleneck       : None
brotli           

### 한글 UnicoedEncodingError를 방지하기 위해 기본 인코딩을 "utf-8"로 설정

In [ ]:
import os
os.environ["PYTHONIOENCODING"] = "utf-8"

### 경고메시지 표시 안하게 설정하기

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

# 3부. 토픽모델링 : LDA 기반 토픽 모델링

## 1. 데이터 준비

### 1-1) 'description' 컬럼 추출

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/데이터/빅데이터개론/final_review_dataset.csv", encoding_errors='ignore')
df.drop(["Unnamed: 0"], inplace = True, axis = 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
description = df['message']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip3 install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 27.6 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt

okt = Okt()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1-2) 형태소 토큰화 : 명사만 추출

In [ ]:
description_noun_tk = []

for d in description:
    description_noun_tk.append(okt.nouns(d)) #형태소가 명사인 것만 추출

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
description_noun_tk2 = []

for d in description_noun_tk:
    item = [i for i in d if len(i) > 1]  #토큰의 길이가 1보다 큰 것만 추출
    description_noun_tk2.append(item)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(description_noun_tk2)

[['선물', '주시', '편지', '색상', '색상', '립스틱', '뚜껑', '립스틱', '과정', '위로', '신분', '립스틱', '주변', '현상', '참고', '선물', '백화점', '약간', '향기', '피하', '크게', '의치', '선택', '선물', '백화점', '생각', '평점', '손등', '좀더', '색상', '카메라', '화질', '립색', '벨벳', '레드카펫', '느낌', '립스틱', '거리', '단점'], ['우선', '포장', '롯데', '택배', '진짜', '배송', '백화점', '화장품', '명품', '롯데', '계속', '구매', '사이트', '가격', '혜택', '백화점', '직송', '만해', '가을', '뮤트', '오버', '트리', '무브', '스테이', '큐리', '어스', '모카', '구매', '테스트', '걱정', '오버', '진짜', '인기', '설트리무브', '모카', '질감', '달라', '누디', '메이크업', '원래', '가지', '칠리', '레이디', '데인', '발라', '파우더', '키스', '라인', '차이', '매트', '표현', '남고', '라인', '페인트', '물감', '발색'], ['제품', '각질', '부각', '입술', '주름', '사이사이', '발색', '매트', '립스틱', '차이점', '왼쪽', '레트로', '매트', '립스틱', '오른쪽', '파우더', '키스', '립스틱', '질감', '파우더', '매트', '조금', '마무리', '파우더', '키스', '립스틱', '브릭', '쓰루', '장밋빛', '코랄', '브릭', '방울', '색감', '브릭', '색상', '발색', '코랄', '브릭', '표현', '제품', '사계절', '내내', '사용', '실제', '색상', '보지', '인터넷', '발색샷', '보고', '그냥', '레트로', '매트', '립스틱', '런웨이', '히트', '살몬', '베이지', '코랄', '색상', '거의', '두운', '피부', '베이지', '

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2. LDA 토픽 모델 구축

### 2-1) LDA 모델의 입력 벡터 생성

In [ ]:
# 최초 한번만 설치
!pip3 install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
import gensim.corpora as corpora

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred

#### (1) 단어 사전 생성

In [ ]:
dictionary = corpora.Dictionary(description_noun_tk2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(dictionary[1])  #작업 확인용 출력

과정


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### (2) 단어와 출현빈도(count)의 코퍼스 생성

In [ ]:
corpus = [dictionary.doc2bow(word) for word in description_noun_tk2]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
corpus

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 4),
  (8, 2),
  (9, 1),
  (10, 3),
  (11, 1),
  (12, 3),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1)],
 [(8, 2),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 2),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 2),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1)],
 [(7, 6),
  (10, 3),
  (38, 1),
  (45, 4),
  (52, 1),
  (53, 2),
  (65, 1),
  (69, 2),
  (73, 3),
  (76, 1),
  (79, 1),
  (80, 1)

### 2-2) LDA 모델 생성 및 훈련

k= 4

In [ ]:
k = 4  #토픽의 개수 설정

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = k, id2word=dictionary, passes=15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model.print_topics(num_topics = k, num_words = 10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.056*"발림" + 0.047*"지속" + 0.040*"색상" + 0.038*"입술" + 0.029*"발색" + 0.029*"색도" + 0.025*"틴트" + 0.020*"각질" + 0.017*"성도" + 0.015*"매트"'),
 (1,
  '0.040*"컬러" + 0.033*"입술" + 0.026*"틴트" + 0.021*"느낌" + 0.018*"색상" + 0.016*"제품" + 0.016*"레드" + 0.016*"핑크" + 0.015*"사용" + 0.014*"발색"'),
 (2,
  '0.026*"진짜" + 0.017*"입술" + 0.017*"구매" + 0.016*"핑크" + 0.016*"느낌" + 0.016*"사진" + 0.013*"틴트" + 0.013*"발색" + 0.013*"쿨톤" + 0.011*"생각"'),
 (3,
  '0.049*"구매" + 0.034*"배송" + 0.032*"제품" + 0.028*"사용" + 0.023*"색상" + 0.021*"가격" + 0.016*"선물" + 0.014*"주문" + 0.012*"포장" + 0.012*"마음"')]

## 3. LDA 토픽 분석 결과 시각화

### 3-2) 토픽 분석 결과 시각화 : pyLDAvis

In [ ]:
 #최초 한번만 설치
!pip3 install pyLDAvis

In [ ]:
import pyLDAvis.gensim

lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.display(lda_vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(lda_vis, "./립제품_vis.html")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


k=3

In [ ]:
k = 3  #토픽의 개수 설정

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics = k, id2word=dictionary, passes=15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model.print_topics(num_topics = k, num_words = 10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.038*"입술" + 0.026*"컬러" + 0.024*"틴트" + 0.022*"느낌" + 0.020*"색상" + 0.019*"발색" + 0.017*"핑크" + 0.012*"지속" + 0.011*"착색" + 0.011*"발라"'),
 (1,
  '0.049*"구매" + 0.035*"색상" + 0.035*"발림" + 0.029*"제품" + 0.027*"색도" + 0.026*"배송" + 0.021*"사용" + 0.020*"지속" + 0.018*"틴트" + 0.018*"가격"'),
 (2,
  '0.022*"립스틱" + 0.022*"선물" + 0.013*"포장" + 0.013*"립밤" + 0.011*"제품" + 0.011*"오렌지" + 0.011*"마스크" + 0.011*"매트" + 0.010*"친구" + 0.010*"사용"')]

In [ ]:
import pyLDAvis.gensim

lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.display(lda_vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(lda_vis, "./립제품_vis(2).html")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
